In [1]:
import os
from PIL import Image
import torch
from torchvision import transforms
import numpy as np

# Define paths
image_dir = '/content/images'  # Update this with your actual image folder path

# Define the transformation for resizing and normalizing images
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load images from directory
def load_images_from_folder(folder):
    images = []
    filenames = []
    for filename in os.listdir(folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Only load .jpg and .png files
            image_path = os.path.join(folder, filename)
            try:
                image = Image.open(image_path).convert('RGB')
                images.append(transform(image))
                filenames.append(filename)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
    return images, filenames

images, image_filenames = load_images_from_folder(image_dir)


In [2]:
import torchvision.models as models

# Load a pre-trained ResNet model
model = models.resnet50(pretrained=True)
model.eval()

# Function to extract features from a single image
def extract_features(image_tensor):
    with torch.no_grad():
        features = model(image_tensor.unsqueeze(0))  # Add batch dimension
    return features.squeeze().numpy()

# Extract features for each image
features = [extract_features(image) for image in images]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 139MB/s]


In [6]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

# Determine the number of components for PCA based on dataset size
n_samples, n_features = np.array(features).shape
n_components = min(20, n_samples, n_features)  # Adjust this value if needed

# Apply PCA
pca = PCA(n_components=n_components)
pca_features = pca.fit_transform(features)

# Apply t-SNE on the PCA-reduced features
tsne = TSNE(n_components=2, perplexity=4, random_state=0)
reduced_embeddings = tsne.fit_transform(pca_features)



In [8]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 35.8 MB/s eta 0:00:00


In [9]:
from dash import Dash, html, dcc
import plotly.express as px

app = Dash(__name__)

# Plot the 2D embeddings
fig = px.scatter(
    x=reduced_embeddings[:, 0],
    y=reduced_embeddings[:, 1],
    text=image_filenames,  # Use filenames as labels for each point
    title="2D Visualization of Image Embeddings"
)

# Dash layout
app.layout = html.Div([
    html.H1("Image Data Embeddings Visualization"),
    dcc.Graph(id='scatter-plot', figure=fig)
])

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>